# House Price Prediction with PySpark

In [107]:
import os

from pyspark.ml.feature import Imputer

from pyspark.ml import Pipeline

from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType

from pyspark.ml.regression import LinearRegression

from pyspark.ml.feature import VectorAssembler

from pyspark.ml.feature import StandardScaler

import pandas as pd

import numpy as np

from pyspark.sql.functions import log, exp

from pyspark.ml.evaluation import RegressionEvaluator

import pyspark.ml.tuning as tune

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark.catalog.listTables()

[]

In [4]:
# get train and test data set
data_loc = './data'

train_data_base = spark.read.csv(os.path.join(data_loc,'train.csv'), inferSchema=True, header=True, nullValue='NA')
test_data_base = spark.read.csv(os.path.join(data_loc,'test.csv'), inferSchema=True, header=True, nullValue='NA')

## some examples

In [5]:
train_data_base.describe().toPandas()

,summary,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,count,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
1,mean,730.5,56.897260273972606,None,70.04995836802665,10516.828082191782,None,None,None,None,...,2.758904109589041,None,None,None,43.489041095890414,6.321917808219178,2007.8157534246575,None,None,180921.19589041095
2,stddev,421.6100093688479,42.30057099381045,None,24.28475177448321,9981.26493237915,None,None,None,None,...,40.17730694453021,None,None,None,496.1230244579441,2.7036262083595113,1.3280951205521145,None,None,79442.50288288663
3,min,1,20,C (all),100,1300,Grvl,Grvl,IR1,Bnk,...,0,Ex,GdPrv,Gar2,0,1,2006,COD,Abnorml,34900
4,max,1460,190,RM,NA,215245,Pave,Pave,Reg,Lvl,...,738,NA,NA,TenC,15500,12,2010,WD,Partial,755000


In [6]:
# add data to the catalog
train_data_base.createOrReplaceTempView("temp")

In [7]:
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [8]:
# run a simple query
spark.sql("SELECT Id, LotFrontage, Alley FROM temp LIMIT 10").show()

+---+-----------+-----+
| Id|LotFrontage|Alley|
+---+-----------+-----+
|  1|         65|   NA|
|  2|         80|   NA|
|  3|         68|   NA|
|  4|         60|   NA|
|  5|         84|   NA|
|  6|         85|   NA|
|  7|         75|   NA|
|  8|         NA|   NA|
|  9|         51|   NA|
| 10|         50|   NA|
+---+-----------+-----+



In [10]:
expensive_houses = train_data_base.select("Id", "LotFrontage", "SalePrice", "YearBuilt", "YrSold")\
    .filter(train_data_base.SalePrice > 500000)

In [11]:
expensive_houses.show(10)

+----+-----------+---------+---------+------+
|  Id|LotFrontage|SalePrice|YearBuilt|YrSold|
+----+-----------+---------+---------+------+
| 179|         63|   501837|     2008|  2009|
| 441|        105|   555000|     2008|  2009|
| 692|        104|   755000|     1994|  2007|
| 770|         47|   538000|     2003|  2010|
| 804|        107|   582933|     2008|  2009|
| 899|        100|   611657|     2009|  2010|
|1047|         85|   556581|     2005|  2006|
|1170|        118|   625000|     1995|  2006|
|1183|        160|   745000|     1996|  2007|
+----+-----------+---------+---------+------+



In [12]:
expensive_houses = expensive_houses.withColumn("AgeSold", expensive_houses.YrSold-expensive_houses.YearBuilt)
expensive_houses.show()

+----+-----------+---------+---------+------+-------+
|  Id|LotFrontage|SalePrice|YearBuilt|YrSold|AgeSold|
+----+-----------+---------+---------+------+-------+
| 179|         63|   501837|     2008|  2009|      1|
| 441|        105|   555000|     2008|  2009|      1|
| 692|        104|   755000|     1994|  2007|     13|
| 770|         47|   538000|     2003|  2010|      7|
| 804|        107|   582933|     2008|  2009|      1|
| 899|        100|   611657|     2009|  2010|      1|
|1047|         85|   556581|     2005|  2006|      1|
|1170|        118|   625000|     1995|  2006|     11|
|1183|        160|   745000|     1996|  2007|     11|
+----+-----------+---------+---------+------+-------+



## ML Model

In [6]:
# find all numerical columns
num_features = [col_name for col_name, dtype in train_data_base.dtypes if dtype == "int"]

# remove SalePrice, which is the target, and the Id column
num_features.remove("SalePrice")
num_features.remove("Id")

In [7]:
# split validation data
training, validation = train_data_base.randomSplit([.7, .3])

In [93]:
# create the train, val and test set, selecting only the numerical columns
train = training.select("*")
val = validation.select("*")
test = test_data_base.select("*")

In [101]:
# cast all numerical features to double (necessary for imputation)
for feat in num_features:
    train = train.withColumn(feat, train[feat].cast(DoubleType()))
    val = val.withColumn(feat, val[feat].cast(DoubleType()))
    test = test.withColumn(feat, test[feat].cast(DoubleType()))

In [96]:
# log-scale the Sale Price
train = train.withColumn("SalePriceLog", log("SalePrice"))
val = val.withColumn("SalePriceLog", log("SalePrice"))

In [97]:
# names of the features afer imputation
num_features_imp = [feat+"_imp" for feat in num_features]

In [98]:
# set up the ML pipeline
imputer = Imputer(inputCols=num_features, outputCols=num_features_imp)

vec_assembler = VectorAssembler(inputCols=num_features_imp, outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

regression = LinearRegression(featuresCol="features_scaled", labelCol="SalePriceLog", elasticNetParam=0)

pipe = Pipeline(stages=[imputer, vec_assembler, scaler, regression])

In [55]:
# create a parameter gird
grid = tune.ParamGridBuilder()
grid = grid.addGrid(regression.regParam, [0.001, 0.01, 0.1, 1, 10, 50, 100, 500, 1000])
grid = grid.addGrid(imputer.strategy, ["mean", "median"])
grid = grid.build()

In [68]:
# define the evaluation criteria
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SalePriceLog", metricName='rmse')

# Create the CrossValidator
cv = tune.CrossValidator(estimator=pipe, estimatorParamMaps=grid, evaluator=evaluator)

# Fit cross validation models
cv_models = cv.fit(train)

# extract the best model
bestPipeline = cv_models.bestModel

In [125]:
# predict and evaluate the validation set
val_prediction = bestPipeline.transform(val)
evaluator.evaluate(val_prediction)

0.18838401725277737

In [104]:
# predict the test set
test_prediction = bestPipeline.transform(test)

In [123]:
submission = test_prediction.select("Id", "prediction")
submission = submission.withColumn("SalePrice", exp("prediction"))
submission = submission.drop("prediction")

In [124]:
submission.write.csv(os.path.join(data_loc, "submission_spark"), header=True)

In [111]:
submission.show()

+----+------------------+
|  Id|        prediction|
+----+------------------+
|1461|123079.74491247827|
|1462|145242.20373664235|
|1463|176763.50538829982|
|1464| 199700.2031702147|
|1465|176019.62870575927|
|1466|172416.71928410057|
|1467|190724.45957536227|
|1468| 165015.4674434123|
|1469|191564.55756314783|
|1470|  114856.393165908|
|1471|191361.16468033532|
|1472|107661.14381960587|
|1473|103158.59752457582|
|1474|154826.98214539504|
|1475|118022.32753484302|
|1476| 321932.9816800341|
|1477| 232822.9661497423|
|1478|269003.95932906197|
|1479| 253977.2554024686|
|1480|474374.49218876363|
+----+------------------+
only showing top 20 rows

